# Collision Avoidance - Train Model(モデルの学習)

Welcome to this host side Jupyter Notebook!  This should look familiar if you ran through the notebooks that run on the robot.  In this notebook we'll train our image classifier to detect two classes
``free`` and ``blocked``, which we'll use for avoiding collisions.  For this, we'll use a popular deep learning library *PyTorch*

[訳] このホスト側のJupyer notebookへようこそ(追加: 今回はJetBot内で学習する)。ロボットの中で実行されるノートブックを実行した場合、これはおなじ感じです。。このnotebookでは、avoiding collisionを使うために、``free``と``blocked``の2つのクラスを特定する画像分類(image classifier)を学習します。

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

### Upload and extract dataset(データセットの解凍をアップロード)

この処理は、ローカルで学習するため、スキップ。

Before you start, you should upload the ``dataset.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot.

You should then extract this dataset by calling the command below

In [ ]:
!unzip -q dataset.zip

You should see a folder named ``dataset`` appear in the file browser.

### Create dataset instance(データセットインスタンスを生成)

Now we use the ``ImageFolder`` dataset class available with the ``torchvision.datasets`` package.  We attach transforms from the ``torchvision.transforms`` package to prepare the data for training.  

(訳) ``torchvision.datasets``パッケージに含まれる``ImageFolder`` dataset classを使用します。学習用のデータを準備するために、``torchvision.transforms``パッケージからtransformsを紐付けます。

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into train and test sets(学習用とテスト用にデータセットを分ける)

Next, we split the dataset into *training* and *test* sets.  The test set will be used to verify the accuracy of the model we train.

(訳) 次に、*training*用と*test*用のセットにデータセットをわけます。テストは、学習したモデルの精度を検証するために使用されます。

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 50, 50])

### Create data loaders to load data in batches(バッチでデータをロードするためのデータローダーの作成)

We'll create two ``DataLoader`` instances, which provide utilities for shuffling data, producing *batches* of images, and loading the samples in parallel with multiple workers.

(訳) 2つの `` DataLoader``インスタンスを作成します。これらのインスタンスは、データのシャッフル、画像の*バッチ*の生成、および複数のワーカーと並行してサンプルをロードするためのユーティリティを提供します


In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define the neural network(Neural networkの定義)

Now, we define the neural network we'll be training.  The *torchvision* package provides a collection of pre-trained models that we can use.

(訳) 学習するためのneural networkを定義します。*torchvision*パッケージで、使用可能な学習済みモデルのコレクションが提供されています。

In a process called *transfer learning*, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.

(訳)*転移学習*と呼ばれる手法で、利用可能なデータがはるかに少ない状態で新しいタスクをおこなうために、事前にトレーニングされたモデル（数百万の画像でトレーニングされた）を再利用する事ができます。

Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  We'll use the ``alexnet`` model.

(訳) 事前にトレーニング済みモデルで学習した重要な機能は、新しいタスクで再利用できます。 今回は、`` alexnet``モデルを使用します

In [ ]:
model = models.alexnet(pretrained=True)

The ``alexnet`` model was originally trained for a dataset that had 1000 class labels, but our dataset only has two class labels!  We'll replace
the final layer with a new, untrained layer that has only two outputs.  

(訳) ``alexnet``モデルは、1000クラスのラベルを持つデータセットを独自に学習させたものですが、我々のデータセットは、2クラスのラベルしか持っていません。
出力が2つしかなく、トレーニングされていない新しいレイヤーを含む最終レイヤーを入れ替えます。

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

Finally, we transfer our model for execution on the GPU

(訳) GPUの実行するために、モデルを転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network(Nueral networkでの学習)

Using the code below we will train the neural network for 30 epochs, saving the best performing model after each epoch.

(訳) 下記コードを使い、30エポック、nueral networkで学習します。それぞれのエポックの後に、最高のパフォーマンスモデルを保存していきます。

> An epoch is a full run through our data.

> エポックとは、データを完全に把握することです。

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

Once that is finished, you should see a file ``best_model.pth`` in the Jupyter Lab file browser.  Select ``Right click`` -> ``Download`` to download the model to your workstation

(訳) 学習が終わると、Jupyer Labのファイルブラウザに``best_model.pth``が生成されます。

ローカルでの学習のためDownload処理は実施しなくて大丈夫です。